<a href="https://colab.research.google.com/github/rackarmattan/DAT405/blob/master/Copy_of_Assignment5_Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DAT405 Introduction to Data Science and AI, 2010-2021, Study Period 2** <br/>
**Assignment 5: Reinforcement learning and Classification** <br/>
**Due Date: Dec 9, 23:59** <br/>

---


**What to submit**
*   **The entire assignment should be submitted through the notebook. No separate file will be accepted.** You can submit either the notebook itself, or a public link to a Google Colab notebook<br/>

*In the notebook:*
*	State your names and how many hours each person spent on the assignment.
*	The solutions and answers to the theoretical and practical problems, including LaTeX math-mode equations, plots and tables etc.
*	All plots/results should be visible such that the notebook does not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.<br/>

*Before submitting:*
*   Make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.

**Self-check**<br/>
Is all the required information included? Have you answered all questions to the best of your ability? Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?) Does your notebook run and can reproduce the results, plots and tables?

**Grading**<br/>
Grading will be based on a qualitative assessment of each assignment. It is important to:
*	Present clear arguments
*	Present the results in a pedagogical way
*	Show understanding of the topics (e.g, write a pseudocode) 
*	Give correct solutions
*	Make sure that the code is well commented 

**Again, as mentioned in general guidelines, all code should be written here. And this same ipython notebook file (Assignment5_Reinforcement_Learning.ipynb) should be submitted with answers and code written in it. No separate file will be accepted.** 


# Primer

## Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

## The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

## Overview
* To make things concrete, we will first focus on decision making under **no** uncertainity, i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.


* Next, we will work through one type of reinforcement learning algorithm called Q-learning. Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

## Markov Decision Process

Markov Decision Process (MDP) provides a mathematical framework for modeling sequential decision making under uncertainty. A MDP consists of five parts: the specific decision times, the state space of the environment/system, the available actions for the decision maker, the rewards, and the transition probabilities between the states.

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ for the current state and action, and the resulting next state
* Transition probabilities $p(s'|s,a)$ that taking action $a$ in state $s$ will lead to state $s'$

At a given decision epoch $t$ and system state $s_t$, the decions maker, or *agent*, chooses an action $a_t$, the system jumps to a new state $s_{t+1}$ according to the transition probability $p(s_{t+1}|s_t,a_t)$, and the agent receives a reward $r_t(s_t,a_t,s_{t+1})$. This process is then repeated for a finite or infinite number of times.

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if you think all future rewards should count equally, you would use $\gamma = 1$, while if you only care less about future rewards you would use $\gamma < 1$. The expected total *discounted* reward becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^{\pi^*}(s)$ for all $s\in S$. That is

$$V^{\pi^*}(s) \geq V^\pi(s), s\in S$$

The problem of finding the optimal policy is a _dynamic programming problem_. It turns out that a solution to the optimal policy problem in this context is the *Bellman equation*. The Bellman equation is given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

Thus, it can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. Your states would be the amount of items you have in stock. Your actions would be the amount to order. The discrete time would be the days of the month. The reward would be the profit.  

A major drawback of MDPs is called the "Curse of Dimensionality". MDPs unfortunately do not scale very well with increasing sets of states or actions.   


## Question 1

In this first question we work with the deterministic MDP, no code is necessary in this part.

Setup:

* The agent starts in state **S**
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* Note, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$

The reward of a state $r(s=(x, y))$ is given by the values on the grid:
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

**Ans:** The optimal path is EENNN, as that is the only path that contains no zeros. Every path from S to F will contain at least two -1's, hence a path with no 0 will be the optimal path as the reward generated is the largest.


**1b)** What is the optimal policy (i.e. the optimal action in each state)?

**Ans:**
1. In state $S_{0,0}$ : do action E
2. In state $S_{1,0}$ : do action E
3. In state $S_{2,0}$ : do action N
4. In state $S_{2,1}$ : do action N
5. In state $S_{2,2}$ : do action N

**1c)** What is expected total reward for the policy in 1b)?

**Ans:** 0


## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation. The process of repeated application of the Bellman equation what we here call the _value iteration_ algorithm.

The value iteration algorithm practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```






**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability of 0.8 that that action will be performed and a probability of 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state, that is, we stay on the grid), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Implement the value iteration algorithm just described here in python, and show the converging optimal value function and the optimal policy for the above 3x3 grid. Hint: use the pseudo-code above as a starting point, but be sure to explain what every line does.

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

In [ ]:
# QUESTION 2
################# HELPER FUNCTIONS #################

# Takes a position, width and height of a grid and return possible actions from that position
# Actions and probability is taken from question 2 description
def get_actions(x, y, width, height):
    all_actions = {
        "W": {
            "pos": (x + -1, y),
            "prob": 0.8
        }, 
        "E": {
            "pos": (x + 1, y),
            "prob": 0.8
        },
        "N": {
            "pos": (x, y + 1),
            "prob": 0.8
        }, 
        "S": {
            "pos": (x, y + -1),
            "prob": 0.8
        }
    }
    if y == height-1:
        # At the top
        del all_actions["N"]
    if y == 0:
        #At the bottom
        del all_actions["S"]
    if x == 0:
        #To the left
        del all_actions["W"]
    if x == width-1:
        #To the right
        del all_actions["E"]

    return all_actions    

# Take a list of actions with rewards and return the action that gives the highest reward 
def get_max_action(l):
    current_action = None
    max_reward = -1000000
    for el in l:
        reward = el["reward"]
        if (reward > max_reward):
            max_reward = reward
            current_action = el
    return current_action

# Prints given value grid and policy grid
def print_grids(grid = None , p_grid = None):
    print("Value matrix")
    if (grid != None):
        for row in reversed(grid):
            print(row)
    print("\nPolicy matrix")
    if (p_grid != None):
        for row in reversed(p_grid):
            print(row)
    print("\n")

def get_max_value_action(x, y, grid, reward_grid, discount, ):
    # List of all possible actions on given state (x,y)
    possible_actions = get_actions(x, y, len(grid[0]),len(grid))
    # List of all possible next states rewards
    next_state_rewards = []
    # Iterate over possible actions and save all the rewards for possible next states in list
    for action in possible_actions:
        # The probability to succeed on given action
        prob = possible_actions[action]["prob"]
        # Position for next state on given action
        next_state = {"x": possible_actions[action]["pos"][0], "y": possible_actions[action]["pos"][1]}
        # Value on the next state in the current grid on given action 
        next_state_reward = grid[next_state["y"]][next_state["x"]]
        # Value on the current state in the current grid on given action 
        current_state_reward = grid[y][x]
        # Reward given if performing an action an to the next state
        immediate_reward_move = reward_grid[next_state['y']][next_state['x']]
        # Reward given if remaining on in the current state
        immediate_reward_remain = reward_grid[y][x]
        # Calculate reward on performing given action
        reward = prob * (immediate_reward_move + discount * next_state_reward) + (1.0 - prob) * (immediate_reward_remain + discount * current_state_reward)
        # Add current action and reward
        next_state_rewards.append({"action": action, "reward": reward})
    # Return a dict containing the action that resulted with the highest reward
    return get_max_action(next_state_rewards)

In [ ]:
# Performs a value iteration algorithm on a grid v0, reward_grid,action probality and discount factor.
# Returns s Vk(s) matrix with vaules in each state and a matrix with policies
def value_iteration(grid, reward_grid, discount = 1, epsilon = 1e-5,current_iteration = 1):
    # Size variables of the grid
    width = len(grid[0])
    height = len(grid)
    # Temporary grid of zeros to put in new values for the next state rewards
    temp_grid = [[0] * width for _ in range(height)]
    # Variable to keep track if the algorithm should keep iterating or stop and return
    keep_iterate = False

    # Calculate optimal values for all states given the possible actions for each state
    for x in range(width):
        for y in range(height):
            vks = get_max_value_action(x, y, grid, reward_grid, discount)["reward"]
            temp_grid[y][x] = round(vks, 3)
    
    # Check if the new value for each state is good enough to keep iterating or if it has no great impact, 
    # then set keep_iterating to false to stop and return  
    for x in range(width):
        for y in range(height):
            diff = abs(temp_grid[y][x] - grid[y][x])
            if diff > epsilon:
               keep_iterate = True
    # Calculate optimal policy for each state given the optimal value grid and then return the value grid and policy grid   
    if  not keep_iterate:
        policy_grid = [[[]] * width for _ in range(height)]
        for x in range(width):
            for y in range(height):
                pis = get_max_value_action(x, y, grid, reward_grid, discount)["action"]
                policy_grid[y][x] = policy_grid[y][x] + [pis]
                
        return temp_grid, policy_grid
    else:
        current_iteration += 1                    
        return value_iteration(temp_grid, reward_grid, disc, current_iteration)

In [ ]:
# VARIABLE SETUP FOR VALUE ITERATION QEUSTION 2
v0 = [[0] * 3 for _ in range(3)]
reward_grid = [[0, 0, 0],
               [0, 10,0],
               [0, 0, 0]]
# QEUSTION 2, result
grid, policys = value_iteration(grid = v0, reward_grid = reward_grid, discount = 0.9)
print_grids(grid, policys)

Value matrix
[57.884, 64.176, 57.884]
[64.176, 60.354, 64.176]
[57.884, 64.176, 57.884]

Policy matrix
[['E'], ['S'], ['W']]
[['E'], ['W'], ['W']]
[['E'], ['N'], ['W']]




**2b.** As the immediate reward, given by going to a certain state is fixed, it is the reward matrix that will steer the policy. As the discount is only applied on the reward at a certain state, not the immediate reward given by the reward matrix, the rewards given from the states get smaller and smaller as the iterations keep going. 

## Reinforcement Learning (RL)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(a,s,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(a,s,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(a,s
]\,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

## Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there is sufficient exploration. While a constant $\alpha$ generally does not guarantee us to reach true convergence, we keep it constant at $\alpha=0.1$ for this assignment.

## OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular OpenAI Gym library. It just implements [different types of simulators](https://gym.openai.com/) including ATARI games. Although here we will only focus on simple ones, such as the [Chain enviroment](https://gym.openai.com/envs/NChain-v0/) illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3
You are to first familiarize with the framework using its [documentation](http://gym.openai.com/docs/), and then implement the Q-learning algorithm for the Chain enviroment (called 'NChain-v0') using default parameters. Finally print the $Q^*$ table at convergence. Convergence is **not** a constant value, rather a stable plateau with some noise. Take $\gamma=0.95$. You can refer to the Q-learning (frozen lake) Jupyter notebook shown in class, uploaded on Canvas. Hint: start with a small learning rate.

## Question 4

**4a)** Define the MDP corresponding to the Chain environment above and verify that the optimal $Q^*$ value obtained using simple Q-learning is the same as the optimal value function $V^*$ for the corresponding MDP's optimal action. Hint: compare values obtained using value iteration and Q-learning.

**4b)** What is the importance of exploration in RL? Explain with an example.


In [ ]:
# QUESTION 3
import gym
import numpy as np
import random
import math

env = gym.make("NChain-v0") 

# Five states and two acitons qives a 5*2 matrix
q_table = np.zeros((5, 2))

# Discount factor
disc = 0.95

# Learning rate
learn_rate = 0.1

# The probability of exploring
epsilon = 0.5

# The no iterations
num_iterations=100
for i in range(num_iterations):
    # Reset the environment at the beginning
    s = env.reset()
    done = False
    
    # Step through environment
    while not done:
        # Choose a random action if a random number is lower than epsilon, allows exploration
        if np.random.random() < epsilon or np.sum(q_table[s, :]) == 0:
            action = np.random.randint(0,2)
        
        # Choose the action that has the highest Q value in the table at the current state
        else:
            action = np.argmax(q_table[s, :])
        
        # Take a step in the environment using the chosen action
        new_state, reward, done, info = env.step(action)
        # Add the Q value generated by this aciton at this step at the action and step position in the Q table
        q_table[s, action] += reward + learn_rate * (disc*np.max(q_table[new_state, :]) - q_table[s, action])
        
        s = new_state
        
print("  Action 0    Action 1")
print(q_table)
env.close()


  Action 0    Action 1
[[646.06023989 645.05679235]
 [682.95414742 651.09926918]
 [734.97705094 684.4357969 ]
 [802.31911557 727.4988229 ]
 [851.01207327 694.67939914]]


In [ ]:
# QUESTION 4, 
# functions to perform value iteration on the chain problem

# Probability for the actions are taken from this article http://ceit.aut.ac.ir/~shiry/lecture/machine-learning/papers/BRL-2000.pdf page 6 
# 0.8 to move or stay and 0.2 to slip back

# Takes a position, width and height of a grid and return possible actions from that position
def get_actions(x, y, width, height):
    # All available actions in the chain problem graph 
    action = {
        "name": "move",
        "pos": (x + 1, y + 0), 
        "prob": 0.8
    }
    # Remove the move action if the current state is most to the right and add the stay action instead
    if x == (width - 1):
        action = {
            "name": "stay",
            "pos": (x, y), 
            "prob": 0.8
        }
    
    return action   

def get_max_value_action(x, y, grid, reward_grid, discount):
    # Possible action to take given current state (x,y)
    action = get_actions(x, y, len(grid[0]), len(grid))
    # Coordinates for next state with given action
    next_state = action["pos"]
    # Probabilty on succeding with given action
    prob = action["prob"]
    # Value at the next state in the grid on given action
    next_state_reward = grid[next_state[1]][next_state[0]]
    # Value at the first state in (0,0) in the grid
    state_one_reward = grid[0][0]
    # Reward on given action(move gives 0)
    immediate_reward_action = 0
    # Reward on slip back action (going back to state 1)
    immediate_reward_go_back = reward_grid[0][0]   
    # Special case when immidiate reward for the action "stay" is 10
    if action["name"] == "stay":
        immediate_reward_action = reward_grid[next_state[1]][next_state[0]]

    # Calculate optimal policy for each state given the optimal value grid and then return the value grid and policy grid
    reward = prob * (immediate_reward_action + discount * next_state_reward) + (1.0 - prob) * (immediate_reward_go_back + discount * state_one_reward)
    
    # Return a dict containing the action that resulted with the highest reward
    return {"action": action["name"], "reward": reward}



In [ ]:
# QUESTION 4, result

# Initial matrix v0
v0 = [[0, 0, 0, 0, 0, 0]]
# Reward matrix
r =  [[2, 0, 0, 0, 0, 10]]

grid, policys = value_iteration(v0, r, discount = 0.95)
print_grids(grid, policys)


Value matrix
[12.388, 15.057, 18.568, 23.189, 29.269, 37.269]

Policy matrix
[['move'], ['move'], ['move'], ['move'], ['move'], ['stay']]




**4a.** Our value iteration shows that the optimal policy for the Chain Problem is to do action a on every state, i.e. move from 1 to 2, 2 to 3, 3 to 4 and finally 4 to 5, and then stay at 5. This can be seen by the policy printed, which tells us that ‘move’, i.e. “go to right”, should be done 5 times, and then ‘stay’ after that. Our Q-table converges to the same conclusion, that it is the best option to do action 0 (move to the right and then stay at 5) in each state. 

**4b.** If no exploration would be allowed, the agent would always choose the action that generated the highest Q-value in the Q-table at a specific state. If the agent would always do so in the chain example, the agent would always choose to go to 1 in space 1, as that generates the highest Q-value. With permission to explore other alternative actions, the algorithm has the possibility to find ways that could lead to higher rewards in the long-term. It will also explore a lot more of the space-action combinations than if just choosing the next best action every time. 


## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

**5b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification.

**5a.** Decision trees can be displayed as a tree-structured graph, where every node is a problem/question. Every branch out of the node is an answer to the node’s question, and either leads to a new node with a new problem/question, or to a leaf (end-point). The leaves are categories. 

A random forest is a number of decision trees, i.e. a forest containing decision trees. Random forests query all the decision trees and calculate the average from the answers they give. Random forests select random features from some data, generates a decision tree from it, and repeats the process until a sufficient amount of decision trees are created. 

**5b.** In supervised tasks such as classification or regression, the algorithm trains on known x, y pairs, to learn how to map new x values to y values. In reinforced learning, an agent is exploring an environment and tries to maximize the rewards it gets by performing some actions. 



# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf